# JAX中的高级自动微分

> 作者： Vlatimir Mikulik & Matteo Hessel

计算梯度是现代机器学习方法的关键部分。本节涵盖了与现代机器学习相关的自动微分中的一些高级主题。

尽管了解自动微分的工作原理对于大多数情况下使用JAX也不是至关重要，但我们鼓励读者观看这个[视频](https://www.bilibili.com/video/BV1YX4y1G7V3/)来获得更深入的认识。

[`Autodiff`指导手册]()是对JAX后端如何实现这些功能更高级更详细的解释。在JAX中进行大多数操作不需要了解这一点。但是，某些功能（例如[自定义微分]()）依赖于对此的理解，因此如果您需要使用这些解释，则值得一看。

## 导入

In [1]:
import jax
import jax.numpy as jnp

## 高阶求导

JAX的`autodiff`使得计算高阶导数变得容易，因为计算导数的函数本身是可微的。因此，高阶导数与叠加变换一样容易。

我们在单变量情况下说明这一点：

函数 $f(x)=x^3 + 2x^2 - 3x + 1$ 可以被计算为：

In [2]:
f = lambda x: x ** 3 + 2 * x ** 2 - 3 * x + 1
dfdx = jax.grad(f)

对于$f$的高阶求导为：

$$
f'(x)=3x^2+4x-3\\
f''(x)=6x+4\\
f'''(x)=6\\
f^{iv}=0\\
$$

在JAX中计算任何导数都像链接`grad`函数一样容易：

In [3]:
d2fdx = jax.grad(dfdx)
d3fdx = jax.grad(d2fdx)
d4fdx = jax.grad(d3fdx)

当`x=1`时，我们将会得到：

$$
f'(1)=4\\
f''(1)=10\\
f'''(1)=6\\
f^{iv}(1)=0\\
$$

使用JAX：

In [4]:
print(dfdx(1.))
print(d2fdx(1.))
print(d3fdx(1.))
print(d4fdx(1.))

4.0
10.0
6.0
0.0


在多变量情况下，高阶导数更为复杂。 函数的二阶导数由其[Hessian矩阵](https://baike.baidu.com/item/黑塞矩阵/2248782)表示，根据:

$$
(\mathbb{H}f)_{i,j}=\frac{\partial^2 f}{\partial_i \partial_j}
$$

多变量实值函数的Hessian，$f:\mathbb{R}^n \rightarrow \mathbb{R}$，可以用其梯度的雅克比矩阵来识别。JAX提供了两种变换来计算雅克比矩阵，即`jax.jacfwd`和 `jax.jacrev`，分别对应于正向和反向模式的`autodiff`。虽然他们给出的答案是相同的，但是在不同情况下，某种方法的效率会更高。请参阅上面的[视频链接](https://www.bilibili.com/video/BV1YX4y1G7V3/)来获得更多解释。

In [5]:
def hessian(f):
    return jax.jacfwd(jax.grad(f))

我们再次检查点乘是否正确： $f:x \rightarrow x^Tx$

如果 $i=j$ , $\frac{\partial^2 f}{\partial_i \partial_j}(x)=2$ 。否则， $\frac{\partial^2 f}{\partial_i \partial_j}(x)=0$ 。

In [6]:
def f(x):
    return jnp.dot(x, x)

hessian(f)(jnp.array([1., 2., 3.]))

DeviceArray([[2., 0., 0.],
             [0., 2., 0.],
             [0., 0., 2.]], dtype=float32)

但是，通常我们对完整的Hessian本身不感兴趣，因此计算效率可能非常低。 [Autodiff指导手册]()解释了一些技巧，例如Hessian-vector乘法，可以在不具体化整个矩阵的情况下使用它。

如果您打算在JAX中使用高阶导数，我们强烈建议您阅读[Autodiff指导手册]()。

## 高阶优化

一些元学习技术，例如与模型无关的元学习（MAML），需要通过梯度更新来区分。 在其他框架中，这可能非常麻烦，但是在JAX中，它要容易得多：

In [7]:
def meta_loss_fn(params, data):
    """
    Computes the loss after one step of SGD.
    在SGD一步后计算损失
    """
    grads = jax.grad(loss_fn)(params, data)
    return loss_fn(params - lr * grads, data)

#meta_grads = jax.grad(meta_loss_fn)(params, data)

## 梯度停止

`Autodiff`使函数能够自动计算相对于输入的梯度。但有时我们可能需要一些其他控制,我们可能希望避免通过计算图的某些子集向后传播梯度。

例如`TD(0)`（时差）强化学习更新。这用于从环境交互的经验中学习估计环境中状态的值。假设由状态 $s_{t-1}$ 中的值来估计 $v_\theta(s_{t-1})$ 由线性函数设定参数。

In [8]:
# Value function and initial parameters
# 值函数和参数初始化
value_fn = lambda theta, state: jnp.dot(theta, state)
theta = jnp.array([0.1, -0.1, 0.])

考虑从状态 $s_{t-1}$ 到 状态 $s_t$，在此过程中我们观察到奖励 $r_t$

In [9]:
s_tm1 = jnp.array([1., 2., -1.])
r_t = jnp.array(1.)
s_t = jnp.array([2., 1., 0.])

网络参数的TD(0)更新为： $$\triangle \theta = (r_t + v_\theta (s_t) - v_\theta (s_{t-1})) \triangledown v_\theta (s_{t-1})$$

此更新不是任何损失函数的梯度。但是它可以写成伪损失函数的梯度：

$$
L(\theta) = [r_t + v_\theta (s_t) - v_\theta(s_{t-1})]^2
$$

如果忽略目标 $r_t + v_\theta(s_t)$对于参数 $\theta$的依赖性。

我们如何在JAX中实现呢？如果我们简单地写出伪损失函数，我们将得到：

In [10]:
def td_loss(theta, s_tm1, r_t, s_t):
    v_tm1 = value_fn(theta, s_tm1)
    target = r_t + value_fn(theta, s_t)
    return (target - v_tm1) ** 2

td_update = jax.grad(td_loss)
delta_theta = td_update(theta, s_tm1, r_t, s_t)

delta_theta

DeviceArray([ 2.4, -2.4,  2.4], dtype=float32)

但`td_update`不会计算TD(0)更新，因为梯度计算将包括目标对 $\theta$的依赖性。
我们可以使用 `jax.lax.stop_gradient` 来强制忽略目标对 $\theta$的依赖：

In [11]:
def td_loss(theta, s_tm1, r_t, s_t):
    v_tm1 = value_fn(theta, s_tm1)
    target = r_t + value_fn(theta, s_t)
    return (jax.lax.stop_gradient(target) - v_tm1) ** 2

td_update = jax.grad(td_loss)
delta_theta = td_update(theta, s_tm1, r_t, s_t)

delta_theta

DeviceArray([-2.4, -4.8,  2.4], dtype=float32)

这会将`target`视为不依赖于参数θ，并计算对参数的正确更新。

`jax.lax.stop_gradient`在其他设置中也可能有用，例如，如果您希望某些损失的梯度仅影响神经网络参数的一个子集（因为其他参数是使用不同的损失函数训练）。

## 使用`stop_gradient`的直通估算器

直通估算器是一种定义不可微分函数梯度的技巧。给定不可微分函数 $f: \mathbb{R}_n \rightarrow \mathbb{R}_n$ 作为我们希望找到其梯度的更大函数的一部分，我们简单地在反向传播过程中假定 `f`是恒等函数。这里可以使用 `jax.lax.stop_gradient` 清楚地实现：

In [13]:
def f(x):
    return jnp.round(x) # 不可微分

def straight_through_f(x):
    return x + jax.lax.stop_gradient(f(x) - x)

print("f(x): ", f(3.2))
print("straight_through_f(x): ", straight_through_f(3.2))

print("grad(f)(x): ", jax.grad(f)(3.2))
print("grad(straight_through_f)(x): ", jax.grad(straight_through_f)(3.2))


f(x):  3.0
straight_through_f(x):  3.0
grad(f)(x):  0.0
grad(straight_through_f)(x):  1.0


## 逐样本梯度

尽管大多数ML系统从一批数据中计算梯度和更新，但出于计算效率和/或方差减少的原因，有时有时必须访问批中每个特定样本相关的梯度/更新。

例如，需要根据梯度大小对数据进行优先级排序，或者对每个样本进行裁剪/归一化。

在许多框架（PyTorch，TF，Theano）中，计算每个示例的梯度通常并不容易，因为该库直接在批处理中累积梯度。简单的解决方法通常效率很低（例如，每个样本计算一个单独的损失，然后汇总所得的梯度）。

在JAX中，我们可以定义代码，以一种简单而有效的方式计算每个样本的梯度。只需将 `jax`， `vmap` 和 `grad` 结合在一起：

In [15]:
perex_grads = jax.jit(jax.vmap(jax.grad(td_loss), in_axes=(None, 0, 0, 0)))

# 测试：
batched_s_tm1 = jnp.stack([s_tm1, s_tm1])
batched_r_t = jnp.stack([r_t, r_t])
batched_s_t = jnp.stack([s_t, s_t])

perex_grads(theta, batched_s_tm1, batched_r_t, batched_s_t)

DeviceArray([[-2.4, -4.8,  2.4],
             [-2.4, -4.8,  2.4]], dtype=float32)

让我们把这些转换一个一个过一遍。

首先，我们将 `jax.grad`应用于`td_loss`以获得一个计算损失地图的函数，也就是单个（非批处理的）输入上的参数：

In [17]:
dtdloss_dtheta = jax.grad(td_loss)
dtdloss_dtheta(theta, s_tm1, r_t, s_t)

DeviceArray([-2.4, -4.8,  2.4], dtype=float32)

该函数计算上面的一行数组。

然后我们使用`jax.vmap`对这个函数进行向量化处理。这会将批次维度添加到所有输入和输出。现在，给定一批输入，我们将产生一批输出——批中的每个输出都对英语输入批中相应成员的梯度。

In [18]:
almost_perex_grads = jax.vmap(dtdloss_dtheta)

batched_theta = jnp.stack([theta, theta])
almost_perex_grads(batched_theta, batched_s_tm1, batched_r_t, batched_s_t)

DeviceArray([[-2.4, -4.8,  2.4],
             [-2.4, -4.8,  2.4]], dtype=float32)

这不是我们最想要的，因为我们必须手动想该函数提供一批 `theta`，而实际上我们只想用一个`theta`。我们通过在`jax.vmap`中